In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive"

'Adsız klasör'	       SydneyHousePrices.csv
 Classes1.py	       VerilerinIncelenmesi-Oğuzhan.ipynb
'Colab Notebooks'      Wine.ipynb
 diabetes.csv	       winemag-data_first150k.csv
 Hitters.csv	      "Wine's Data - PreProcessing.ipynb"
 protein_ortak.ipynb


In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score,confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Hitters.csv")

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pandas as pd 
import seaborn as sns
import missingno as msno
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import shapiro
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from sklearn import model_selection
from warnings import filterwarnings
from sklearn.neural_network import MLPRegressor,MLPClassifier
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import numpy as np 


filterwarnings("ignore")


class Visualization():
    def __init__(self,data):
        self.df = data
    
    def scatter(self,x,y,z = None):
        scatter = sns.scatterplot(x = x, y = y, hue=z,data=self.df)
        return scatter
    
    def hist(self,x):
        plt.figure(figsize=(8,6))
        hist = sns.distplot(self.df[[x]], rug=True, rug_kws={"color":"g"}, kde_kws={"color":"k","lw":3,"label":"KDE"},
                            hist_kws={"histtype":"step","linewidth":"3","alpha":1,"color":"g"})
        
        return hist
    
    def hist1(self,x,y=None):
        hist1 = sns.FacetGrid(self.df,hue=y,height=5).map(sns.kdeplot,x,shade = True).add_legend()
        return hist1
    
    def corr_map(self):
        df = self.df.corr()
        plt.figure(figsize=(8.5,5.5))
        corr = sns.heatmap(df,xticklabels=df.columns,yticklabels=df.columns,annot=True)
        return corr
        
    
    
    def bar(self,x,y,z = None):
        plt.figure(figsize=(8.5,5.5))
        bruhh = sns.barplot(x = x, y = y, hue=z,data=self.df,capsize=.2)
        return bruhh
    
    def boxplot(self,x,y=None,z = None):
        box = sns.boxplot(x = x, y = y, hue=z, data=self.df,linewidth=2.5)
        return box
    
    def heatmap(self):
        pass
    
    def missingvalues_heat_map(self):
        heatmap = msno.heatmap(self.df)
        return heatmap
    
    def missingvalues_dendogram(self):
        
        dendogram = msno.dendrogram(self.df)
        return dendogram


class information():
    def __init__(self,data):
        self.df = data
    
    def info_data(self):
        """İnfo about data"""
        information = self.df.info()
        shape = self.df.shape
        return print(information,"\nBoyut Bilgisi:\n",shape)
    
    def summary_statistics(self):
        info = self.df.describe().T
        return info
    
    def value_count(self,x,y=None):
        """Write X with two quotes"""
        counts = self.df[x].value_counts()
        counts1 = self.df[y].value_counts()
        print("First Variable Value Counts:\n",counts,"\nSecond Variable Value Counts:\n",counts1)
        
    
    def nuniques(self):
        unique = self.df.nunique()
        return unique
    
    def MissingValues(self):
        missing_values = self.df.isnull().sum()
        missing_values_rate = (self.df.isnull().sum() * 100) / self.df.shape[0]
        df_Missing_values = missing_values
        df_Missing_values_rate = missing_values_rate
        table = pd.concat([df_Missing_values,df_Missing_values_rate],axis=1)
        table = table.rename(columns={0:"Missing_Values",1:"Missing_values_rate %"})
        return table
    
class preprocess():
    
    def __init__(self,df):
        self.df = df
    
    def outlier(self):
        
        lower_and_upper = {}
        
        columns = list(self.df.select_dtypes(include=["float64","int64"]))
        
        for col in columns:
            q1 = self.df[col].quantile(0.25)
            q3 = self.df[col].quantile(0.75)
            iqr = 1.5*(q3-q1)
    
            lower_bound = q1-iqr
            upper_bound = q3+iqr
    
            lower_and_upper[col] = (lower_bound, upper_bound)
            self.df.loc[(self.df.loc[:,col]<lower_bound),col]=lower_bound
            self.df.loc[(self.df.loc[:,col]>upper_bound),col]=upper_bound
         
        return self.df
 
    
class statistic():
    def __init__(self,df):
        
        self.df = df

        
    def normal(self):
        
        normal_dagilanlar = []
        normal_dagilmayanlar = []
        columns = list(self.df)
        alpha = 0.05
    
        for col in columns:
            result = shapiro(self.df[[col]])
        
            if result[1] < alpha:
                normal_dagilmayanlar.append(col)
            
            elif result[1] > alpha:
                normal_dagilanlar.append(col)
            
        
        print("Normal dagilimdan gelenler: ",normal_dagilanlar,
    
        "\nNormal dagilimdan gelmeyenler: \n",normal_dagilmayanlar)
        
        
        
    def yakala(self,df,Y):
        anlamli_kat_sayilar = []
        target = Y
        columns = list(df.drop(target,axis=1))
        sonuclar = []
        for col in columns:
        
            X = df[[col]]
            y = df[target]
            lm = sm.OLS(y,X)
            model = lm.fit()
            if (model.tvalues.values > 1.96):
                anlamli_kat_sayilar.append(model.tvalues.index)
        
            elif (model.tvalues.values < -1.96):
                anlamli_kat_sayilar.append(model.tvalues.index)
            
            
        for i in anlamli_kat_sayilar:
        
            print("Anlamli Kat sayilar: ",i[0])


class Methods():

    def __init__(self):
        """Methods of machine learning"""

    

    
    
    def LogitReg(self,x_train,y_train,x_test,y_test):
        loj = sm.Logit(y_train,x_train)
        loj_model = loj.fit()
        result = loj_model.summary()
    
   
        solver = input("Please enter any method for solver: ")
    
        loj1 = LogisticRegression(solver = solver) # Liblinear yerine baska yontemler var bunun bunlar da data yapisina gore sonuclar vermekte
        loj_model1 = loj1.fit(x_train,y_train)
        loj_model1
        intercept = loj_model1.intercept_
        coef_sklearn = loj_model1.coef_
    
    
        print("OLS Results: \n",result)
        print("\nSklearn Results of intercept and coef: \n")
        print("Intercept: \n",intercept)
        print("Coef: \n",coef_sklearn)
        print("------------------------------------")
        print("------------------------------------")
        print("------------------------------------")
        print("OLS and Sklearn result are diffrent beacuse OLS result haven't Intercept")
        
        
        
        
    def LojPredict(self,x_train,y_train,x_test,y_test):
        
        print("this func shows us Acuary value for both train and test set\nThen Same kind Confusion Matirx and result details\nWe will see Cross_val values each sets\nAt the end we'll see  ROC CURVE with made of TRAIN set  ")
        
        solver = input("Please enter any method for solver: ")
    
        loj1 = LogisticRegression(solver = solver) # Liblinear yerine baska yontemler var bunun bunlar da data yapisina gore sonuclar vermekte
        loj_model1 = loj1.fit(x_train,y_train)
    
        y_pred = loj_model1.predict(x_test)
    
        accuary = accuracy_score(y_test,y_pred)
        confusion = confusion_matrix(y_test,y_pred)
        result_detail = classification_report(y_test,y_pred)
    
    
        logit_roc_auc = roc_auc_score(y_train,loj_model1.predict(x_train))
        fpr, tpr, tresholds = roc_curve(y_train,loj_model1.predict_proba(x_train)[:,1])

        ROC = plt.figure(),
        plt.plot(fpr, tpr, label = "AUC (area = %0.2f)"% logit_roc_auc),
        plt.plot([0,1], [0,1], "r--"),
        plt.xlim([0.0, 1.0]),
        plt.ylim([0.0,1.05]),plt.xlabel("False Positive Orani"),
        plt.ylabel("Ture Positive Orani"),
        plt.title("ROC")
    
    
        cross_val_train = cross_val_score(loj_model1, x_train, y_train, cv = 10).mean()
        cross_val_test = cross_val_score(loj_model1, x_test, y_test, cv = 10).mean()

    
    
        print("Accuary : ",accuary)
        print("-------------------")
        print("Confusion Matrix: \n",confusion)
        print("-------------------")
        print("Details: \n",result_detail)
        print("-------------------")
        print("Cross Validaiton About TRAIN and TEST set: \n",cross_val_train," \n",cross_val_test)
        print("-------------------")
        print("ROC CURVE: \n",ROC)
    
        
        
        
    def LogitSelf(self,x_train,y_train,x_test,y_test):
    
        print("This func job is if you given a TRESHOLD value it will use that and shows new results\n")
    
        solver = input("Please enter any method for solver: ")
        threshold = float(input("Please enter TRESHOLD value: "))
    
        loj1 = LogisticRegression(solver = solver) 
        loj_model1 = loj1.fit(x_train,y_train)
    
        y_probs = loj_model1.predict_proba(x_train)
        y_probs = y_probs[:,1]
        y_pred = [1 if i > threshold else 0 for i in y_probs]
    
        accuary = accuracy_score(y_train,y_pred)
        confusion = confusion_matrix(y_train,y_pred)
        result_detail = classification_report(y_train,y_pred)
    
    
        print("Accuary : ",accuary)
        print("-------------------")
        print("Confusion Matrix: \n",confusion)
        print("-------------------")
        print("Details: \n",result_detail)
        
        


    def PcaReg(self,x_train,y_train,x_test,y_test):
        pcr = PCA()
        pca = PCA()
        X_reduced_train = pcr.fit_transform(scale(x_train))
    
        bilesen_yuzde = np.cumsum(np.round(pcr.explained_variance_ratio_, decimals = 4)*100)[:]
    
        features = range(pcr.n_components_)
    
        #bilesen_gorsel = plt.bar(features,pcr.explained_variance_ratio_,color = "red"),plt.xlim(0,20),
        #plt.xlabel("PCA Bilesenler"),plt.ylabel("Aciklanan Varyans %"),plt.xticks(features)
    
        bilesen_gorsel = plt.figure(figsize=(25,8)),plt.bar(features,pcr.explained_variance_ratio_,color = "red"),plt.xlim(0,25),
        plt.xlabel("PCA Bilesenler"),plt.ylabel("Aciklanan Varyans %"),plt.xticks(features)
    
    
    
        lm = LinearRegression()

        pcr_model = lm.fit(X_reduced_train[:,:], y_train)  

        sm1 = sm.OLS(y_train,X_reduced_train[:,:])
        pcr_model_v  = sm1.fit()
    
    
        print("Bilesen sayisina gore aciklanabilirlik: \n",bilesen_yuzde)
        print("------------------------------------------")
        print("Bilesen sayisina gore aciklanma durumu",bilesen_gorsel)
        print("------------------------------------------")
        print("\nTrain setine ait sabit Kat sayi: \n",pcr_model.intercept_,"\nTrain setine ait modelin kat sayilari: \n",pcr_model.coef_)
        print("OLS sonuclari : \n",pcr_model_v.summary().tables[0]) 

        
        
    def PcaPredict(self,x_train,y_train,x_test,y_test):
        pcr = PCA()
        X_reduced_train = pcr.fit_transform(scale(x_train))
    
        lm = LinearRegression()
    
    
    
    
        pcr_model = lm.fit(X_reduced_train[:,:], y_train)  
    
    
        y_pred = pcr_model.predict(X_reduced_train[:,:]) 
        train_result = np.sqrt(mean_squared_error(y_train,y_pred))
    
        pcr1 = PCA()

        X_reduced_test = pcr1.fit_transform(scale(x_test))

        y_pred1 = pcr_model.predict(X_reduced_test[:,:]) # yine 4 bilesen 


        test_result = np.sqrt(mean_squared_error(y_test,y_pred1))
        train_r2 = r2_score(y_train,y_pred)
    
        cv_10 = model_selection.KFold(n_splits=10,
                             shuffle=True,
                             random_state=1)
    
    
    
        RMSE = []

        for i in np.arange(1, X_reduced_train.shape[1]+1):
        
            score = np.sqrt(-1*model_selection.cross_val_score(lm,
                                                          X_reduced_train[:,:i],
                                                          y_train.ravel(),
                                                          cv = cv_10,
                                                          scoring = "neg_mean_squared_error").mean())
    
            RMSE.append(score)
    
    
    
    

        Degerlendirme = plt.plot(RMSE, "-v"),
        plt.xlabel("Bilesen sayisi"),plt.ylabel("RMSE"),
        plt.title("Tahmin Modeli Icin Pcr Model Tune");
    
    
    
        print("Train setinin RMSE degeri: \n",train_result,"\nTest setinin RMSE degeri: \n",test_result)
        print("---------------------------------------------------------------------------")
        print("R2 degeri: ",train_r2)
        print("---------------------------------------------------------------------------")
        print("Bilesenlere gore RMSE durumu cross validation ile: \n",Degerlendirme)
        
        
        
        

    def YsaClassifier(self,X_train,y_train,X_test,y_test):
        scaler = StandardScaler()

        X_train_scale = scaler.fit_transform(X_train)

        X_test_scale =  scaler.fit_transform(X_test)
  
  
        mlp_regres = MLPClassifier().fit(X_train_scale,y_train) 
        y_pred = mlp_regres.predict(X_test_scale)
        Accuaracy = accuracy_score(y_test,y_pred)
        matrix = confusion_matrix(y_test,y_pred)
  # tune edelim

        params = {"alpha":[0.1,0.01,0.02,0.005], # alpha icin aranacak degerler
              "hidden_layer_sizes":[(20,20),(100,50,150),(300,200,150)], # gizli katmanin dereceleri ve sayilari icin aranacak parametreler
              "activation":["relu","logistig"],
              'solver': ['adam', 'lbfgs']}# Son olarak birde iki tane fonksiyon var onlari denesin denedik 

        mlp_c = MLPClassifier()
  
        mlp_c = GridSearchCV(mlp_c,params,
                       cv = 10,
                       n_jobs = -1,
                       verbose = 2)

        mlp_c_tune = mlp_c.fit(X_train_scale,y_train)

        bos = []
        for i in mlp_c_tune.best_params_:
            bos.append(mlp_c_tune.best_params_[i])

        mlp_tuned = MLPClassifier(activation=bos[0],
                         alpha=bos[1],hidden_layer_sizes=bos[2],
                         solver=bos[3]).fit(X_train_scale,y_train)

        y_pred1 = mlp_tuned.predict(X_test_scale)

        Accuaracy1 = accuracy_score(y_test,y_pred1)
        matrix1 = confusion_matrix(y_test,y_pred1)


        print("Tune Edilmemis Tahmin sonuclari Accuracy degeri: ",Accuaracy)
        print("-------------------------------")
        print("Tune Edilmemis Confusion matrix sonuc: \n",matrix)
        print("****************************************************")
        print("Tune sonrasi Tahmin sonuclari RMSE degeri:",Accuaracy1)
        print("-------------------------------")
        print("Tune sonrasi Confusion matrix sonuc: \n",matrix1)
        
        
        


    def YsaReg(self,X_train,y_train,X_test,y_test):
        
        scaler = StandardScaler()

        X_train_scale = scaler.fit_transform(X_train)

        X_test_scale =  scaler.fit_transform(X_test)
  
  
        mlp_regres = MLPRegressor().fit(X_train_scale,y_train) 
        y_pred = mlp_regres.predict(X_test_scale)
        rmse = np.sqrt(mean_squared_error(y_test,y_pred))

  # tune edelim

        params = {"alpha":[0.1,0.01,0.02,0.005], # alpha icin aranacak degerler
             "hidden_layer_sizes":[(20,20),(100,50,150),(300,200,150)], # gizli katmanin dereceleri ve sayilari icin aranacak parametreler
             "activation":["relu","logistig"],
              'solver': ['adam', 'lbfgs'],
              'learning_rate': ['constant','adaptive']}# Son olarak birde iki tane fonksiyon var onlari denesin denedik 

        mlp_cv_model = GridSearchCV(mlp_regres,params,cv = 10,
                       n_jobs = -1,
                       verbose = 2)

        mlp_cv_model = mlp_cv_model.fit(X_train_scale,y_train)

        bos = []
        for i in mlp_cv_model.best_params_:
            bos.append(mlp_cv_model.best_params_[i])

        mlp_tuned = MLPRegressor(activation=bos[0],
                         alpha=bos[1],hidden_layer_sizes=bos[2],
                         learning_rate=bos[3],
                         solver=bos[4]).fit(X_train_scale,y_train)

        y_pred = mlp_tuned.predict(X_test_scale)

        new_rmse = np.sqrt(mean_squared_error(y_test,y_pred))


        print("Tune Edilmemis Tahmin sonuclari RMSE degeri: ",rmse)
        print("---------------------------------------------")
        print("Tune sonrasi Tahmin sonuclari RMSE degeri:",new_rmse)
        
        

In [ ]:
import Classes1 as cl

In [ ]:
!ls "/content/drive/My Drive"

'Adsız klasör'	    protein_ortak.ipynb
 Classes1.py	    SydneyHousePrices.csv
'Colab Notebooks'   VerilerinIncelenmesi-Oğuzhan.ipynb
 diabetes.csv	    Wine.ipynb
 Hitters.csv	    winemag-data_first150k.csv
 hmelq.csv	   "Wine's Data - PreProcessing.ipynb"
 HW_Data_Set.xlsx


In [ ]:
df = pd.read_csv("/content/drive/My Drive/hmelq.csv")
df.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
0,0,81200,18834.0,108355.0,HomeImp,NaN,28.0,0.0,0.0,139.14,0.0,14.0,34.042
1,0,12600,103960.0,127384.0,DebtCon,NaN,2.0,0.0,0.0,129.02,0.0,25.0,34.479
2,0,18000,46865.0,61266.0,DebtCon,NaN,5.0,0.0,0.0,102.59,2.0,9.0,26.354
3,0,10300,57676.0,71027.0,DebtCon,NaN,19.0,0.0,0.0,157.52,1.0,11.0,33.992
4,0,9400,56508.0,78358.0,DebtCon,NaN,17.0,0.0,0.0,141.93,0.0,11.0,32.327


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bad,3576.0,0.199385,0.399594,0.00,0.00,0.000,0.00,1.000
loan,3576.0,18529.697987,11262.834060,1100.00,11000.00,16200.000,23100.00,89900.000
mortdue,3262.0,74013.907992,44016.833772,2619.00,46700.25,65858.000,91717.00,399550.000
value,3512.0,101076.922759,54865.275351,8800.00,65753.00,89183.000,118928.25,850000.000
yoj,3264.0,8.832644,7.568046,0.00,3.00,7.000,13.00,41.000
derog,3149.0,0.261670,0.865178,0.00,0.00,0.000,0.00,10.000
delinq,3225.0,0.432558,1.099230,0.00,0.00,0.000,0.00,15.000
clage,3397.0,179.832529,87.066198,0.49,115.60,173.640,230.71,1168.230
ninq,3273.0,1.186373,1.728723,0.00,0.00,1.000,2.00,17.000
clno,3443.0,21.507116,10.167490,0.00,15.00,20.000,26.00,71.000


In [ ]:
df.isnull().sum()

bad          0
loan         0
mortdue    314
value       64
reason     147
job        167
yoj        312
derog      427
delinq     351
clage      179
ninq       303
clno       133
debtinc    767
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bad,2018.0,0.088206,0.283665,0.00,0.0000,0.000,0.0000,1.000
loan,2018.0,19010.257681,10768.607345,1700.00,12000.0000,16900.000,23600.0000,89900.000
mortdue,2018.0,76337.035183,44476.368189,5076.00,49577.2500,67595.000,92922.7500,399412.000
value,2018.0,106899.244797,54026.707096,23031.00,71287.2500,94514.000,121220.7500,512650.000
yoj,2018.0,9.003964,7.573012,0.00,3.0000,7.000,13.0000,41.000
derog,2018.0,0.150149,0.580421,0.00,0.0000,0.000,0.0000,10.000
delinq,2018.0,0.264123,0.789816,0.00,0.0000,0.000,0.0000,10.000
clage,2018.0,180.645020,84.420116,0.49,118.2050,176.670,229.6925,1168.230
ninq,2018.0,1.037661,1.578414,0.00,0.0000,0.000,2.0000,13.000
clno,2018.0,22.265114,9.457238,0.00,16.0000,21.000,27.0000,64.000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 153 to 3549
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bad      2018 non-null   int64  
 1   loan     2018 non-null   int64  
 2   mortdue  2018 non-null   float64
 3   value    2018 non-null   float64
 4   reason   2018 non-null   object 
 5   job      2018 non-null   object 
 6   yoj      2018 non-null   float64
 7   derog    2018 non-null   float64
 8   delinq   2018 non-null   float64
 9   clage    2018 non-null   float64
 10  ninq     2018 non-null   float64
 11  clno     2018 non-null   float64
 12  debtinc  2018 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 220.7+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder

df.dropna(inplace=True)

lbe = LabelEncoder()
df["reason"] = lbe.fit_transform(df["reason"])
df = pd.get_dummies(df,columns= ["job"], prefix= ["job"])

In [ ]:
df.head(2)

,bad,loan,mortdue,value,reason,yoj,derog,delinq,clage,ninq,clno,debtinc,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,0,18200,94727.0,136877.0,0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,0,0,0,0
154,0,21700,79240.0,96784.0,0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,0,0,0,0


In [ ]:
y = df["bad"]
X = df.drop(labels=["bad"],axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state = 42,
                                                    test_size = 0.25)

In [ ]:
YSA = cl.Methods()

YSA.YsaClassifier(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.3min finished


Tune Edilmemis Tahmin sonuclari Accuracy degeri:  0.9465346534653465
-------------------------------
Tune Edilmemis Confusion matrix sonuc: 
 [[457   9]
 [ 18  21]]
****************************************************
Tune sonrasi Tahmin sonuclari RMSE degeri: 0.9663366336633663
-------------------------------
Tune sonrasi Confusion matrix sonuc: 
 [[460   6]
 [ 11  28]]


In [ ]:
df["ninq"].value_counts()

0.0     1014
1.0      480
2.0      278
3.0      140
4.0       40
5.0       15
6.0       14
7.0       11
10.0      10
8.0        8
9.0        6
13.0       1
11.0       1
Name: ninq, dtype: int64

In [ ]:
a = df["ninq"] == 0
df["ninq"][a] = "a"

a = df["ninq"] == 1
df["ninq"][a] = "b"
a = df["ninq"] == 2
df["ninq"][a] = "b"
a = df["ninq"] == 3
df["ninq"][a] = "b"

a = df["ninq"] == 4
df["ninq"][a] = "c"
a = df["ninq"] == 5
df["ninq"][a] = "c"
a = df["ninq"] == 6
df["ninq"][a] = "c"

a = df["ninq"] == 7
df["ninq"][a] = "d"
a = df["ninq"] == 8
df["ninq"][a] = "d"
a = df["ninq"] == 9
df["ninq"][a] = "d"

a = df["ninq"] == 10
df["ninq"][a] = "e"
a = df["ninq"] == 11
df["ninq"][a] = "e"
a = df["ninq"] == 12
df["ninq"][a] = "e"
a = df["ninq"] == 13
df["ninq"][a] = "e"

In [ ]:
df["ninq"].value_counts()

a    1014
b     898
c      69
d      25
e      12
Name: ninq, dtype: int64

In [ ]:
df = pd.get_dummies(df,columns= ["ninq"], prefix= ["ninq"])

In [ ]:
df.head(2)

,bad,loan,mortdue,value,reason,yoj,derog,delinq,clage,clno,debtinc,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self,ninq_a,ninq_b,ninq_c,ninq_d,ninq_e
153,0,18200,94727.0,136877.0,0,15.0,0.0,0.0,168.96,26.0,36.056,1,0,0,0,0,0,0,1,0,0,0
154,0,21700,79240.0,96784.0,0,5.0,0.0,0.0,64.51,24.0,38.079,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
y = df["bad"]
X = df.drop(labels=["bad"],axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state = 42,
                                                    test_size = 0.25)


YSA = cl.Methods()

YSA.YsaClassifier(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.6min finished


Tune Edilmemis Tahmin sonuclari Accuracy degeri:  0.9405940594059405
-------------------------------
Tune Edilmemis Confusion matrix sonuc: 
 [[457   9]
 [ 21  18]]
****************************************************
Tune sonrasi Tahmin sonuclari RMSE degeri: 0.9524752475247524
-------------------------------
Tune sonrasi Confusion matrix sonuc: 
 [[457   9]
 [ 15  24]]


In [ ]:
df_y = pd.read_excel("/content/drive/My Drive/HW_Data_Set.xlsx")
df_y.head(2)

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,ind_17,ind_18,ind_19,ind_20,ind_21,ind_22,ind_23,ind_25,ind_26,ind_27,ind_28,ind_29,ind_30,ind_31,ind_32,ind_33,ind_34,ind_35,ind_36,ind_37,ind_39,ind_40,ind_41,ind_42,ind_43,ind_44,ind_45,ind_46,ind_47,ind_48,...,ind_311,ind_313,ind_315,ind_316,ind_318,ind_321,ind_324,ind_327,ind_330,ind_332,ind_334,ind_338,ind_340,ind_344,ind_345,ind_347,ind_349,ind_351,ind_352,ind_387,ind_392,ind_397,ind_399,ind_403,ind_404,ind_406,ind_407,ind_410,ind_412,ind_414,ind_416,ind_418,ind_420,ind_422,ind_424,ind_426,ind_428,20_target,50_target,90_target
0,19,17,100.0,85.714286,14.285714,72.363515,60.808814,23.80,17.62,11.73,12.060000,9.1100,2,15,13,1,56,0.002690,-0.002670,0.000010,66.666667,93.692449,146.592674,150.505806,154.486989,158.009516,142.421114,142.023350,196.592517,184.819232,31.939605,3,30.752568,-5,-22,-22,-21,-23,-36,-40,...,97.980028,1.179209,-0.756946,98.905563,1.190348,99.156641,-0.000062,12.056738,-0.157039,20.967742,-0.000091,1.197238,-0.319064,0.0,0.0,0.0,0.0,0.0,0.0,-3.731343,-32.587430,-26.386973,-0.003323,0.045130,-0.044678,-44,-23,-27,0.002371,2.0,-49.6,-54,-152,-353,1.0,0.498547,0.701906,15.135802,35.625252,36.997753
1,24,19,100.0,78.571429,21.428571,74.275883,64.366798,11.45,18.16,12.22,11.326667,9.2225,0,11,11,1,63,0.002729,-0.003111,-0.000191,66.666667,76.694104,140.227205,156.903278,161.604303,169.720843,152.634492,149.634976,201.738617,191.985663,38.478501,3,36.812130,-6,-18,-24,-24,-25,-38,-43,...,97.932319,1.179223,-0.756569,98.859627,1.190389,99.111914,-0.000062,11.971831,-0.156961,20.800000,-0.000091,1.197328,-0.318905,0.0,0.0,0.0,0.0,0.0,0.0,-1.369863,-38.115965,-26.085949,-0.000476,0.035314,-0.034961,-48,-16,-31,0.003074,0.8,-55.6,-60,-158,-359,1.0,0.537088,0.690833,15.143348,35.643013,37.016198


In [ ]:
df_y.loc[df_y[df_y["ind_420"] == "?"].index] = np.nan 

# Degisken donusturmeden once icerisinde NAN degerler oldugu icin ilk once bu OBJECT tipindeki degiskenlerin NAN degerlerini degistiriyoruz
# Ardindan donusum gercelestiriyoruz

df_y["ind_422"] = df_y["ind_422"].fillna(df_y.mean()["ind_422"])
df_y["ind_420"] = df_y["ind_420"].fillna(df_y.mean()["ind_420"])

df_y["ind_422"] = df_y["ind_422"].astype('int32')
df_y["ind_420"] = df_y["ind_420"].astype('int32')

# Geriye kalan NaN degiskenleri de dolduruyoruz

column = list(df_y.select_dtypes(include=["int32","float64"]))
df_y[column] = df_y[column].apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 136 entries, ind_5 to 90_target
dtypes: float64(133), int32(2), object(1)
memory usage: 6.4+ MB


In [ ]:
df_y = pd.get_dummies(df_y,columns= ["ind_109"], prefix= ["ind_109"])

df_y = df_y.drop(["ind_109_RED"],axis=1)

df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 136 entries, ind_5 to ind_109_GREEN
dtypes: float64(133), int32(2), uint8(1)
memory usage: 6.3 MB


In [ ]:
df_y.drop(labels=["20_target","90_target"],axis=1,inplace=True)


X = df_y.drop(labels="50_target",axis=1)
y = df_y["50_target"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 42,shuffle = False)

YSA = cl.Methods()

YSA.YsaReg(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 63.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 84.3min finished


Tune Edilmemis Tahmin sonuclari RMSE degeri:  45.60909466505888
---------------------------------------------
Tune sonrasi Tahmin sonuclari RMSE degeri: 42.63284271311603
